# Hello World: Content-Addressed Knowledge Substrate

This notebook demonstrates two substrate-level properties described in
[*A Content-Addressed Adaptive Knowledge Substrate for Distributed Epistemic Coordination*](joven_knowledge_substrate.md) (Joven, 2026):

1. **Content-addressed change detection** — if a fact node's content changes, its hash changes.
   Unchanged nodes retain hashes and are recognizable as unchanged by hash comparison alone.

2. **Scoped recomputation of derived conclusions** — a cached derivation is keyed by the hashes
   of the facts it depends on. After a perturbation, the system invalidates only derivations
   whose dependency hashes are no longer present, then recomputes just those.

This connects to the LLM reasoning failure taxonomy in
[Song et al. (2026)](https://arxiv.org/abs/2602.06176) — specifically:

| Failure category (Song et al.) | Substrate mechanism demonstrated here |
|-------------------------------|---------------------------------------|
| Working memory / proactive interference | Knowledge externalized into persistent graph; updates produce new hashes, not token competition |
| Multi-agent termination | Fixed-point convergence: cycle ends when root hash stabilizes |
| Redundant recomputation | Tier 1 cache check avoids re-deriving conclusions whose inputs haven't changed |

This is intentionally minimal. It does not claim to eliminate hallucination or solve
compositional reasoning in general. It demonstrates deterministic invalidation and
recomputation boundaries in a typed, content-addressed substrate.

## Primitives

Three building blocks: typed nodes, content-addressed hashing, and typed edges.

In [ ]:
import hashlib
import json
from dataclasses import dataclass, field
from enum import Enum
from typing import Optional


class NodeType(Enum):
    ENTITY = "entity"
    RELATION = "relation"
    DERIVED = "derived"


class EdgeType(Enum):
    SUBJECT = "subject"
    OBJECT = "object"
    DEPENDS = "depends"


@dataclass
class Node:
    """Content-addressed node. Hash is deterministic over type + content + dependency hashes."""
    node_type: NodeType
    content: dict
    dependency_hashes: tuple[str, ...] = field(default_factory=tuple)
    _hash: Optional[str] = field(default=None, repr=False)

    @property
    def hash(self) -> str:
        if self._hash is None:
            payload = json.dumps(
                {
                    "type": self.node_type.value,
                    "content": self.content,
                    "deps": list(self.dependency_hashes),
                },
                sort_keys=True,
                separators=(",", ":"),
            )
            self._hash = hashlib.sha256(payload.encode("utf-8")).hexdigest()
        return self._hash

    def __repr__(self) -> str:
        label = self.content.get("label", self.content)
        return f"Node({self.node_type.name}, {label}, {self.hash[:12]}...)"


print("Primitives loaded.")

## Simulated LLM

A deterministic stand-in for an LLM that performs simple transitive reasoning
over fact nodes. Tracks call count and token usage so we can measure
the cost difference between full recomputation and scoped recomputation.

In [ ]:
class SimulatedLLM:
    def __init__(self):
        self.call_count = 0
        self.total_tokens = 0
        self.call_log: list[dict] = []

    def reason_over_subgraph(self, facts: list[dict], query: str) -> dict:
        self.call_count += 1

        prompt_tokens = sum(len(json.dumps(f, separators=(",", ":"))) for f in facts) + len(query)
        completion_tokens = 60
        total = prompt_tokens + completion_tokens
        self.total_tokens += total

        entities: dict[str, str] = {}
        relations: list[dict] = []
        for f in facts:
            if f.get("type") == "entity":
                entities[f["id"]] = f["name"]
            elif f.get("type") == "relation":
                relations.append(f)

        relations = sorted(relations, key=lambda r: r.get("order", 0))

        if len(relations) >= 2:
            r1, r2 = relations[0], relations[1]
            conclusion = (
                f"By transitivity: {entities[r1['subject']]} {r1['predicate']} {entities[r1['object']]}, "
                f"and {entities[r2['subject']]} {r2['predicate']} {entities[r2['object']]}. "
                f"Therefore {entities[r1['subject']]} {r1['predicate']} {entities[r2['object']]}."
            )
            hops = 2
        elif len(relations) == 1:
            r = relations[0]
            conclusion = f"Direct: {entities[r['subject']]} {r['predicate']} {entities[r['object']]}."
            hops = 1
        else:
            conclusion = "Insufficient facts for derivation."
            hops = 0

        result = {"conclusion": conclusion, "confidence": 0.95, "reasoning_hops": hops}

        self.call_log.append({
            "query": query,
            "input_facts": len(facts),
            "prompt_tokens": prompt_tokens,
            "completion_tokens": completion_tokens,
            "total_tokens": total,
        })
        return result


print("SimulatedLLM loaded.")

## Substrate

The substrate holds content-addressed nodes, typed edges, a root hash committing
to the full graph state, a delta chain recording state transitions, and a
derivation cache keyed by dependency hashes.

Key operations:
- **`commit`** — compute root hash over nodes + edges, append to delta chain
- **`is_cached_derivation_valid`** — Tier 1 check: are all dependency hashes still present?
- **`derive`** — Tier 3: invoke the LLM, cache the result keyed by dependency hashes

In [ ]:
class Substrate:
    def __init__(self, llm: SimulatedLLM):
        self.nodes: dict[str, Node] = {}
        self.edges: list[tuple[str, str, str]] = []  # (src, dst, edge_type)
        self.root_hash: Optional[str] = None
        self.delta_chain: list[dict] = []
        self.derived_cache: dict[str, Node] = {}
        self.llm = llm

    def add_node(self, node: Node) -> str:
        h = node.hash
        self.nodes.setdefault(h, node)
        return h

    def add_edge(self, src_hash: str, dst_hash: str, edge_type: EdgeType) -> None:
        self.edges.append((src_hash, dst_hash, edge_type.value))

    def remove_node(self, node_hash: str) -> None:
        """Remove a node and all edges referencing it."""
        self.nodes.pop(node_hash, None)
        self.edges = [
            (s, d, t) for s, d, t in self.edges
            if s != node_hash and d != node_hash
        ]

    def compute_root_hash(self) -> str:
        """Root hash commits to BOTH nodes and edges — graph state, not just node multiset."""
        node_hashes = sorted(self.nodes.keys())
        edges = sorted(self.edges)
        payload = json.dumps({"nodes": node_hashes, "edges": edges}, separators=(",", ":"))
        return hashlib.sha256(payload.encode("utf-8")).hexdigest()

    def commit(self, label: str = "") -> dict:
        new_root = self.compute_root_hash()
        delta = {
            "label": label,
            "prev_root": self.root_hash,
            "new_root": new_root,
            "changed": new_root != self.root_hash,
            "node_count": len(self.nodes),
            "edge_count": len(self.edges),
        }
        self.delta_chain.append(delta)
        self.root_hash = new_root
        return delta

    def is_cached_derivation_valid(self, query_key: str) -> bool:
        """Tier 1: O(k) hash-presence check. No serialization, no model call."""
        cached = self.derived_cache.get(query_key)
        if cached is None:
            return False
        return all(dep in self.nodes for dep in cached.dependency_hashes)

    def derive(self, query: str, fact_hashes: list[str], query_key: str) -> Node:
        """Tier 3: invoke LLM over a subgraph, cache result keyed by dependency hashes."""
        facts = []
        for h in fact_hashes:
            node = self.nodes[h]
            facts.append({"type": node.node_type.value, **node.content})

        result = self.llm.reason_over_subgraph(facts, query)

        derived = Node(
            node_type=NodeType.DERIVED,
            content={"query": query, "label": result["conclusion"], **result},
            dependency_hashes=tuple(sorted(fact_hashes)),
        )
        self.add_node(derived)
        self.derived_cache[query_key] = derived
        return derived


print("Substrate loaded.")

---

## Phase 1: Build a knowledge graph and derive a conclusion

We create a simple 2-hop management chain: Alice manages Bob, Bob manages Carol.
Then we ask the substrate to derive who Alice transitively manages.

This is the baseline: one LLM call, full cost.

In [ ]:
llm = SimulatedLLM()
substrate = Substrate(llm)

# Entities
alice = Node(NodeType.ENTITY, {"id": "alice", "name": "Alice", "label": "Alice", "type": "entity"})
bob   = Node(NodeType.ENTITY, {"id": "bob",   "name": "Bob",   "label": "Bob",   "type": "entity"})
carol = Node(NodeType.ENTITY, {"id": "carol", "name": "Carol", "label": "Carol", "type": "entity"})

alice_h = substrate.add_node(alice)
bob_h   = substrate.add_node(bob)
carol_h = substrate.add_node(carol)

# Relations
rel1 = Node(NodeType.RELATION, {
    "id": "r1", "label": "Alice manages Bob",
    "subject": "alice", "object": "bob",
    "predicate": "manages", "order": 0, "type": "relation",
})
rel2 = Node(NodeType.RELATION, {
    "id": "r2", "label": "Bob manages Carol",
    "subject": "bob", "object": "carol",
    "predicate": "manages", "order": 1, "type": "relation",
})

rel1_h = substrate.add_node(rel1)
rel2_h = substrate.add_node(rel2)

# Edges
substrate.add_edge(rel1_h, alice_h, EdgeType.SUBJECT)
substrate.add_edge(rel1_h, bob_h, EdgeType.OBJECT)
substrate.add_edge(rel2_h, bob_h, EdgeType.SUBJECT)
substrate.add_edge(rel2_h, carol_h, EdgeType.OBJECT)

delta = substrate.commit("initial")

print(f"Nodes: {len(substrate.nodes)}  Edges: {len(substrate.edges)}")
print(f"Root:  {substrate.root_hash[:16]}...")
print()
for h, n in substrate.nodes.items():
    print(f"  {n}")

In [ ]:
# Derive a transitive conclusion (Tier 3 — LLM call)
query = "Who does Alice transitively manage through the chain of command?"
query_key = "transitive_management"
all_fact_hashes = [alice_h, bob_h, carol_h, rel1_h, rel2_h]

derived1 = substrate.derive(query, all_fact_hashes, query_key)
substrate.commit("derived_1")

phase1_calls = llm.call_count
phase1_tokens = llm.total_tokens

print(f"Conclusion: {derived1.content['conclusion']}")
print(f"LLM calls:  {phase1_calls}")
print(f"Tokens:     {phase1_tokens}")
print(f"Root:       {substrate.root_hash[:16]}...")

---

## Phase 2: Perturb one fact, observe scoped invalidation

We replace Carol with Dave in hop-2 of the management chain.

The substrate detects that the cached derivation's dependency hashes no longer
all resolve — `carol_h` and `rel2_h` are gone. This is a **Tier 1** check:
O(k) hash lookups, no serialization, no model call.

Only after invalidation does the system invoke a **Tier 3** recomputation
over the updated subgraph.

This maps to the preprint's tiered operation cost model (§2.3):
- Tier 1 (trivial): hash-presence check → flags the stale derivation
- Tier 3 (expensive): LLM call → runs only because Tier 1 confirmed it's needed

In [ ]:
# Remove old hop-2 (node + edges cleaned up together)
substrate.remove_node(rel2_h)
substrate.remove_node(carol_h)

# Add new hop-2
dave = Node(NodeType.ENTITY, {"id": "dave", "name": "Dave", "label": "Dave", "type": "entity"})
dave_h = substrate.add_node(dave)

rel2_new = Node(NodeType.RELATION, {
    "id": "r2", "label": "Bob manages Dave",
    "subject": "bob", "object": "dave",
    "predicate": "manages", "order": 1, "type": "relation",
})
rel2_new_h = substrate.add_node(rel2_new)

substrate.add_edge(rel2_new_h, bob_h, EdgeType.SUBJECT)
substrate.add_edge(rel2_new_h, dave_h, EdgeType.OBJECT)

substrate.commit("perturb")

print(f"Root after perturbation: {substrate.root_hash[:16]}...")
print(f"Nodes: {len(substrate.nodes)}  Edges: {len(substrate.edges)}")

In [ ]:
# Tier 1: check if cached derivation is still valid
valid = substrate.is_cached_derivation_valid(query_key)
print(f"Cached derivation valid? {valid}")

if not valid:
    # Tier 3: recompute only the invalidated derivation
    fact_hashes2 = [alice_h, bob_h, dave_h, rel1_h, rel2_new_h]
    derived2 = substrate.derive(query, fact_hashes2, query_key)
    substrate.commit("derived_2")
    print(f"Recomputed: {derived2.content['conclusion']}")
else:
    print("Cache hit — no recomputation needed.")

phase2_calls = llm.call_count - phase1_calls
phase2_tokens = llm.total_tokens - phase1_tokens

print(f"\nPhase 2 LLM calls: {phase2_calls}")
print(f"Phase 2 tokens:    {phase2_tokens}")

---

## Fixed-Point Convergence

From the preprint (§2.5): *"A reasoning cycle terminates naturally when running
another cycle would produce the same root hash."*

After Phase 2, no new facts have arrived and the derivation cache is fresh.
Committing again produces the same root hash — the system is at a fixed point.

In [ ]:
pre_root = substrate.root_hash
substrate.commit("fixed_point_check")
post_root = substrate.root_hash

print(f"Root before: {pre_root[:16]}...")
print(f"Root after:  {post_root[:16]}...")
print(f"Fixed point: {pre_root == post_root}")

---

## Cost Summary

In a naive system without content-addressing, Phase 2 would re-derive *all*
conclusions from scratch (cold-start overhead — preprint §6). Here, the
Tier 1 hash check costs nothing, and only the one invalidated derivation
triggers a Tier 3 LLM call.

With N cached derivations and 1 perturbation, naive cost is O(N) LLM calls.
Substrate cost is O(k) hash lookups + O(invalidated) LLM calls.

In [ ]:
print("=== Cost Summary ===")
print(f"Phase 1:  {phase1_calls} LLM call(s), {phase1_tokens} tokens")
print(f"Phase 2:  {phase2_calls} LLM call(s), {phase2_tokens} tokens")
print(f"Total:    {llm.call_count} LLM call(s), {llm.total_tokens} tokens")
print()
print("=== LLM Call Log ===")
for i, entry in enumerate(llm.call_log):
    print(f"  [{i}] facts={entry['input_facts']} tokens={entry['total_tokens']}")

---

## Delta Chain

The full history of state transitions. From the preprint (§2.4):
*"Any state in the history is reconstructible from the delta chain;
the substrate always knows what it knew at any prior moment."*

In [ ]:
print("=== Delta Chain ===")
for i, d in enumerate(substrate.delta_chain):
    print(
        f"  [{i}] {d['label']:20s}  changed={str(d['changed']):5s}"
        f"  nodes={d['node_count']}  edges={d['edge_count']}"
        f"  root={d['new_root'][:12]}..."
    )